# Б1 Кластеризация
    выполнил Сергей Харитонов МИВТ-221

# Задание
Разбить заданный датасет на 5 кластеров и на 2 кластера, используя агломеративный иерархический метод и метод к-средних;
Сравнить результаты;
Данные выбрать из папки data (ссылка на нее приложена) в соответствии с файлом "список заданий".

# Вариант
sales_data.csv
Поля:
- flag:   Была ли совершена покупка
- gender: Пол
- education: Образование
- house_val: Стоимость дома
- age: Возрастная группа
- online: Был ли опыт онлайн покупок
- customer_psy: Переменная, описывающая психологию потребителя в зависимости от района проживания (что бы это ни было)
- marriage: Семейный статус
- child: Есть ли дети
- occupation: Информация о карьере покупателя
- mortgage: Информация о кредите на жилье
- house_owner: В какой собственности дом
- region: Регион проживания
- car_prob: Вероятность того, что клиент купит новый автомобиль
- fam_income: Информация о семейном доходе клиента (A означает самый низкий, а L - самый высокий)


In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from sklearn.cluster import KMeans, AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
import matplotlib.pyplot as plt

# Константы для визуализации кластеров
CLUSTERS = [
    # (color, sign)
    ('lightgreen', 's'),
    ('orange', 'o'),
    ('lightblue', 'v'),
    ('fuchsia', '*'),
    ('black', 'd'),
    ('blue', 's'),
    ('pink', 'o'),
    ('purple', 'v'),
    ('yellow', '*'),
    ('cyan', 'd'),
]
CENTROIDS = ('red', 'P')

In [2]:
# Загрузка данных в память
data = pd.read_csv('data.csv')

In [3]:
# Вывод информации о датасете
def data_analise(_data: DataFrame):
    _data.info()
    _data.describe()
    _data.nunique()
    feature_names = _data.columns.tolist()
    for column in feature_names:
        print(column)
        print(_data[column].value_counts(dropna=False))
        print()


data_analise(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   flag          40000 non-null  object
 1   gender        40000 non-null  object
 2   education     39259 non-null  object
 3   house_val     40000 non-null  int64 
 4   age           40000 non-null  object
 5   online        40000 non-null  object
 6   customer_psy  40000 non-null  object
 7   marriage      25973 non-null  object
 8   child         40000 non-null  object
 9   occupation    40000 non-null  object
 10  mortgage      40000 non-null  object
 11  house_owner   36623 non-null  object
 12  region        40000 non-null  object
 13  car_prob      40000 non-null  int64 
 14  fam_income    40000 non-null  object
dtypes: int64(2), object(13)
memory usage: 4.6+ MB
flag
Y    20000
N    20000
Name: flag, dtype: int64

gender
M    22019
F    16830
U     1151
Name: gender, dtype: int64

educa

In [4]:
# мапинги преобразования данных
Y_N_mapping = {
    'Y': 1,
    'N': 0
}

letter_mapping = lambda letter: (ord(letter) - ord('A') + 1) if letter != 'U' else 0

get_first_number = lambda value: int(value[0])
value_or_nan = lambda mapping, nan_value: (
    lambda value: mapping(value)
    if type(value) is str
    else nan_value
)

mapping = {
    'flag': Y_N_mapping,
    'gender': {
        'M': 2,
        'F': 1,
        'U': 0
    },
    # NaN приравнивается к коледжу, т.к. у большей части именно такое образование
    'education': value_or_nan(get_first_number, 2),
    'age': get_first_number,
    'online': Y_N_mapping,
    'customer_psy': letter_mapping,
    'marriage': value_or_nan(lambda value: {'Married': 2, 'Single': 0}[value], 1),
    'child': {
        **Y_N_mapping,
        '0': 0,
        'U': 2
    },
    'occupation': {
        'Professional': 1,
        'Sales/Service': 2,
        'Blue Collar': 3,
        'Retired': 4,
        'Others': 5,
        'Farm': 6,
    },
    'mortgage': get_first_number,
    'house_owner': value_or_nan(lambda value: {'Renter': 2, 'Owner': 1}[value], 0),
    'region': {
        'South': 1,
        'West': 2,
        'Midwest': 3,
        'Northeast': 4,
        'Rest': 5,
    },
    'fam_income': letter_mapping,
}

In [5]:
# Приведение датасета в необходимое состояние
for column, column_mapping in mapping.items():
    data[column] = data[column] \
        .map(column_mapping)
# Нормализация
data = (data - data.min()) / (data.max() - data.min())

In [6]:
data_analise(data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   flag          40000 non-null  float64
 1   gender        40000 non-null  float64
 2   education     40000 non-null  float64
 3   house_val     40000 non-null  float64
 4   age           40000 non-null  float64
 5   online        40000 non-null  float64
 6   customer_psy  40000 non-null  float64
 7   marriage      40000 non-null  float64
 8   child         40000 non-null  float64
 9   occupation    40000 non-null  float64
 10  mortgage      40000 non-null  float64
 11  house_owner   40000 non-null  float64
 12  region        40000 non-null  float64
 13  car_prob      40000 non-null  float64
 14  fam_income    40000 non-null  float64
dtypes: float64(15)
memory usage: 4.6 MB
flag
1.0    20000
0.0    20000
Name: flag, dtype: int64

gender
1.0    22019
0.5    16830
0.0     1151
Name: gender, dt

In [7]:
data.describe()
print(data)

       flag  gender  education  house_val       age  online  customer_psy  \
0       1.0     1.0       1.00   0.075646  0.000000     0.0           0.2   
1       0.0     0.5       0.75   0.021317  1.000000     0.0           0.5   
2       0.0     1.0       0.50   0.011115  0.166667     1.0           0.3   
3       1.0     1.0       0.50   0.035415  0.166667     1.0           0.2   
4       1.0     0.5       0.50   0.011709  0.000000     1.0           1.0   
...     ...     ...        ...        ...       ...     ...           ...   
39995   1.0     0.5       0.75   0.000000  1.000000     1.0           0.3   
39996   0.0     0.5       0.25   0.021360  0.500000     0.0           0.9   
39997   1.0     1.0       0.00   0.013407  0.333333     1.0           0.6   
39998   0.0     1.0       0.25   0.040221  1.000000     1.0           0.5   
39999   0.0     0.5       0.75   0.083603  1.000000     1.0           0.2   

       marriage  child  occupation  mortgage  house_owner  region  car_prob

In [8]:
clustered_data_with_flag = data.assign(cluster_all=
                                        KMeans(
                                            n_clusters=2,
                                            init='random',
                                            n_init=10,
                                            random_state=0
                                        ).fit_predict(data)
                                        )
clustered_data_without_flag = data.assign(cluster_all=
                                        KMeans(
                                            n_clusters=2,
                                            init='random',
                                            n_init=10,
                                            random_state=0
                                        ).fit_predict(data.iloc[: , 1:])
                                        )

In [9]:

def accuracy(data: DataFrame):
    TP_plus_TN = len(data[data.flag == data.cluster_all])
    FP_plus_FN = len(data[data.flag != data.cluster_all])
    return TP_plus_TN / (FP_plus_FN + TP_plus_TN)

In [10]:
print(f'Точность классификации с колонкой флаг {accuracy(clustered_data_with_flag)}')
print(f'Точность классификации без колонки флаг {accuracy(clustered_data_without_flag)}')

Точность классификации с колонкой флаг 0.9995
Точность классификации без колонки флаг 0.599575


In [11]:
clustered_data_with_flag.to_csv('clustered_data_with_flag.csv', index=False)
clustered_data_without_flag.to_csv('clustered_data_without_flag.csv', index=False)